# Import Libraries

In [53]:
%%capture
#!pip install --upgrade fastapi ffmpeg uvicorn python-multipart tensorflow-gpu scikit-image imutils wandb tensorflow_hub Pillow pyyaml

from fastapi import FastAPI, File, UploadFile
from skimage.metrics import structural_similarity as compare_ssim

from collections import Counter

import tensorflow_hub as hub
import numpy as np
import tensorflow as tf
from PIL import Image
import ffmpeg, shutil

from google.cloud import storage
import nest_asyncio, uvicorn, os, pathlib
import yaml
import cv2, wandb

# Set Up Google Cloud Parameters


## Set Up Google Cloud Project and Model Location

In [59]:
location = 'james-mlsys' # Model Storage Bucket

## Create Storage Bucket

In [60]:
storage_client = storage.Client() #.from_service_account_json('gcp-key.json')

bucket = storage_client.bucket(location)

## Double Check Cloud Bucket (Development Code Only)

In [62]:
%%capture
blobs = storage_client.list_blobs(location)
for blob in blobs:
    print(blob.name)

# WandB Functions

In [104]:
def init_wandb(project_name):
   global wandb_project
   wandb_project = str(project_name)
   wandb.init(project=wandb_project, sync_tensorboard=True)
   return True

# Set File Location

In [64]:
def set_folder_location(in_location):
    global location 
    location = str(in_location)
    global bucket
    bucket = storage_client.bucket(location)
    return True

# Split Video to Frames and Upload

## Download Video to Local Instance

In [65]:
def download_video(video_name):
    print("Downloading: " + str(video_name))
    blob = bucket.blob(video_name)
    blob.download_to_filename(video_name)

## Break down video to frames

In [66]:
def split_video_frames(video_name):
    print("Splitting: " + str(video_name))
    folder = video_name[:-4]
    try:
        shutil.rmtree(str(folder))
    except:
        pass
    os.mkdir(str(folder))
    
    video_capture = cv2.VideoCapture(str(video_name))
    saved_frame_name = 1

    while True:
        print("Frame: " + format(saved_frame_name, '05d'), end="\r")
        success, frame = video_capture.read()

        if success:
            cv2.imwrite(f"{str(folder)}/frame{format(saved_frame_name, '05d')}.png", frame)
            saved_frame_name += 1
        else:
            break
    print("Done                       ")

## Upload Video Frames

In [67]:
def upload_frames_from_folder(folder_name):
    files=sorted(os.listdir(str(folder_name)))
    #files=files[1:]
    
    print("Uploading Frames")
    for i in range(len(files)):
        print(files[i] + "             ", end="\r")
        blob = bucket.blob(folder_name + "/" + files[i])
        blob.upload_from_filename(folder_name + "/" + files[i])
        
    print("Done Uploading               ", end="\r")

## SSIM Compare Video Frames for Novel Frames

## Remove Blurry Images from Set

### Calculate Blurriness using Laplacian

In [68]:
def variance_of_laplacian(image):
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()

### Remove Blurry Images

In [69]:
def remove_blurry_images(folder_name):
    files=sorted(os.listdir(str(folder_name)))
    files=files[1:]
    
    blurriness = np.zeros(len(files))
    
    print("Calculating Average Blurriness")
    for i in range(len(files)):
        print(files[i] + "             ", end="\r")
        img=cv2.imread(folder_name+'/'+files[i])
        img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blurriness[i]=variance_of_laplacian(img_gray)
        wandb.log({'Individual Laplacian': blurriness[i]})
    
    median_blur = np.median(blurriness)
    min_blur = np.min(blurriness)
    max_blur = np.max(blurriness)
    wandb.log({'Batch Median Laplacian': median_blur})
    print("Median Blur (Laplacian Variance): " + str(median_blur))
    blur_cutoff = median_blur*1.05 #+ ((1-average_blur)*0.1)
    print("Blur Cutoff (Laplacian Variance): " + str(blur_cutoff))
    
    print("Removing Noisy Images")
    
    count = 0
    for i in range(len(files)):
        if blurriness[i] > blur_cutoff:
            #print("Deleting " + files[i] + " - Laplacian Noisiness: " + str(blurriness[i]))
            os.remove(folder_name+'/'+files[i])
            count += 1
    blur_ratio = count/len(files)
    wandb.log({'Noisy Frame Ratio': blur_ratio})
    print("Done Checking Frames                  ")
    return {'Total Original Frames': len(files), 'Removed Blurry Frames': count, 'Median Laplacian Variance': median_blur, 'Minimum Laplacian Variance': min_blur, 'Maximum Laplacian Variance': max_blur, 'Noisy Frame Ratio': blur_ratio}

# Deduplicate Similar Frames

## Calculate Similarity Between Images

In [70]:
def compare_images(image1, image2):
    image_gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image_gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    diff, _ = compare_ssim(image_gray1, image_gray2, full=True)
    return diff

## Remove Duplicates

In [71]:
def remove_duplicates(folder_name):
    files=sorted(os.listdir(str(folder_name)))
    files=files[1:]
    print("Removing Duplicate and Highly Similar Frames\nCalculating Frame Similarities")
    
    diff = np.zeros(len(files)-1)    
    
    for i in range(len(files)-1):
        image1 = cv2.imread(folder_name+'/'+files[i])
        image2 = cv2.imread(folder_name+'/'+files[i+1])
        diff[i] = compare_images(image1, image2)
        wandb.log({'Individual Frame Similarities': diff[i]})
        print(str(diff[i]), end="\r")
    
    median_diff = np.median(diff)
    wandb.log({'Batch Median Frame Similarity': median_diff})
    
    diff_cutoff = median_diff*1.05
    
    if diff_cutoff < 0.95:
        diff_cutoff = 0.95
        
    print("Similarity Cutoff (OpenCV Compare Images): " + str(diff_cutoff))
    print("Removing Duplicate Images")
    
    count = 0
    for i in range(len(diff)):
        if diff[i] > 0.99:
            #print("Deleting " + files[i] + " - Similarity: " + str(diff[i]), end="\r")
            os.remove(folder_name+'/'+files[i])
            wandb.log({'Duplicates Similarity': diff})
            count += 1
        
    duplicate_ratio = count/len(files)
    wandb.log({'Batch Duplicate Remove Ratio': duplicate_ratio})
    print("Done Checking Frames, " + str(count) + " frames removed.")
    return {'Removed Duplicate Frames': count, 'Median Frame Similarity': median_diff, 'Duplicate Frame Ratio': duplicate_ratio}

# Get Insight into the Dataset using Faster RCNN Resnet50 COC0 2018/01/28

In [72]:
model_url = 'http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz'
base_url = os.path.dirname(model_url)+"/"
model_file = os.path.basename(model_url)
model_name = os.path.splitext(os.path.splitext(model_file)[0])[0]
model_dir = tf.keras.utils.get_file(fname=model_name, origin=base_url + model_file, untar=True)
model_dir = pathlib.Path(model_dir)/"saved_model"
model = tf.saved_model.load(str(model_dir))
model = model.signatures['serving_default']

In [73]:
CLASS_LABELS = {1: "person", 2: "bicycle", 3: "car", 4: "motorcycle", 5: "airplane", 6: "bus", 7: "train", 8: "truck", 9: "boat", 10: "traffic light", 11: "fire hydrant", 13: "stop sign", 14: "parking meter", 15: "bench", 16: "bird", 17: "cat", 18: "dog", 19: "horse", 20: "sheep", 21: "cow", 22: "elephant", 23: "bear", 24: "zebra", 25: "giraffe", 27: "backpack", 28: "umbrella", 31: "handbag", 32: "tie", 33: "suitcase", 34: "frisbee", 35: "skis", 36: "snowboard", 37: "sports ball", 38: "kite", 39: "baseball bat", 40: "baseball glove", 41: "skateboard", 42: "surfboard", 43: "tennis racket", 44: "bottle", 46: "wine glass", 47: "cup", 48: "fork", 49: "knife", 50: "spoon", 51: "bowl", 52: "banana", 53: "apple", 54: "sandwich", 55: "orange", 56: "broccoli", 57: "carrot", 58: "hot dog", 59: "pizza", 60: "donut", 61: "cake", 62: "chair", 63: "couch", 64: "potted plant", 65: "bed", 67: "dining table", 70: "toilet", 72: "tv", 73: "laptop", 74: "mouse", 75: "remote", 76: "keyboard", 77: "cell phone", 78: "microwave", 79: "oven", 80: "toaster", 81: "sink", 82: "refrigerator", 84: "book", 85: "clock", 86: "vase", 87: "scissors", 88: "teddy bear", 89: "hair drier", 90: "toothbrush"}

In [74]:
def detect_objects(image):
    img = Image.open(image)
    input_tensor = tf.convert_to_tensor(img)
    input_tensor = input_tensor[tf.newaxis,...]
    output_dict = model(input_tensor)
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() for key,value in output_dict.items()}
    classes = output_dict['detection_classes'].astype(np.int64)
    class_names = [None] * len(classes)
    for i in range(len(classes)):
        class_names[i]=CLASS_LABELS[classes[i]]
    output = Counter(class_names)
    wandb.log({'Detections per Image': num_detections})
    return {'Objects Detected':dict(output), 'Number of Detections': num_detections}

In [75]:
def detect_file(folder_name):
    detect_list = []
    for file in sorted(os.listdir(folder_name)):
        output = {'Frame': file[:-4]}
        total_location = folder_name+"/"+file
        output.update(detect_objects(total_location))
        detect_list.append(output)
    return {'Classification Information': detect_list}

# Full Video Analysis and Upload

In [76]:
def clean_video(video_name):
    video_name = str(video_name)
    folder_name = str(video_name)[:-4]
    yaml_val = {'Video': video_name}
    download_video(video_name)
    split_video_frames(video_name)
    yaml_val.update(remove_blurry_images(folder_name))
    yaml_val.update(remove_duplicates(folder_name))
    yaml_val.update(detect_file(folder_name))
    f = open(folder_name+'/'+folder_name+".yaml", "w")
    yaml.dump(yaml_val, f, default_flow_style=False)
    f.close()
    upload_frames_from_folder(folder_name)
    shutil.rmtree(folder_name)
    os.remove(video_name)

# Image Sequence Analysis

## Download Images

In [166]:
def download_images(folder_name):
    print("Checking if folder already downloaded", end="\r")
    if os.path.exists(folder_name):
        print("Folder " + folder_name + " already downloaded")
        return
    print("Downloading: " + str(folder_name))
    os.mkdir(str(folder_name))
    os.mkdir(str(folder_name)+'/images')
    os.mkdir(str(folder_name)+'/labels')
    os.mkdir(str(folder_name)+'/labels/car')
    os.mkdir(str(folder_name)+'/labels/cyclist')
    os.mkdir(str(folder_name)+'/labels/pedestrian')
    blobs = storage_client.list_blobs(location, prefix = folder_name)
    for blob in blobs:
        filename = blob.name.replace('/', '_') 
        print(filename, end = '\r')
        blob.download_to_filename(filename) 
        new_filename = blob.name[len(folder_name):]
        shutil.move(filename, "kitti" + new_filename)
        #print(blob.name, end="\r")
        #blob.download_to_filename(folder_name)

## Images Workflow

In [167]:
def clean_images(folder_name):
    folder_name = str(folder_name)
    yaml_val = {'Folder': folder_name}
    download_images(folder_name)
    yaml_val.update(remove_blurry_images(folder_name+"/images"))
    yaml_val.update(remove_duplicates(folder_name+"/images"))
    yaml_val.update(detect_file(folder_name))
    f = open(folder_name+'/'+folder_name+".yaml", "w")
    yaml.dump(yaml_val, f, default_flow_style=False)
    f.close()
    upload_frames_from_folder(folder_name)
    shutil.rmtree(folder_name)
    os.remove(video_name)

In [ ]:
clean_images("kitti")

Folder kitti already downloadedloaded
Calculating Average Blurriness


In [ ]:
init_wandb(location)

# Analyze Entire Bucket

In [25]:
def clean_entire_bucket_videos():
    blobs = storage_client.list_blobs(location)
    for blob in blobs:
        clean_video(blob.name)

# FastAPI Deployment

In [26]:
app = FastAPI()

@app.on_event("startup")
def start_wandb():
    init_wandb(location)
    return {'message': ('Weights and Balances Started as project: ' + wandb_project)}

@app.get('/')
def index():
    return {'message': 'This is the homepage of the model, add \'/docs\' to the end of the URL to access FastAPI to make predictions with the model'}

@app.get('/set_gcp_location')
def set_gcp_location(string_input):
    set_folder_location(str(string_input))
    return {'message': ('GCP Location Set to: ' + location)}

@app.get('/clean_single_video')
async def single_clean(string_input):
    clean_video(str(string_input))
    return {'message': ('Video: ' + str(string_input) + ' cleaned and uploaded to gs://' + location + "/" + str(string_input))}

@app.get('/clean_single_video')
async def single_clean(string_input):
    clean_images(str(string_input))
    return {'message': ('Images in folder: ' + str(string_input) + ' cleaned and uploaded to gs://' + location + "/" + str(string_input))}


@app.get('/clean_bucket_video')
async def full_clean_videl():
    clean_entire_bucket_video()
    return {'message': ('Bucket: ' + location + ' cleaned and uploaded to gs://' + location)}

# Main Function

In [ ]:
nest_asyncio.apply()
wandb.login(relogin=True)
uvicorn.run(app, host='0.0.0.0', port=8000)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc
INFO:     Started server process [11658]
INFO:     Waiting for application startup.
wandb: Currently logged in as: jamesysato (use `wandb login --relogin` to force relogin)


INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
